# Homework 2

Let's create a social media account for your agent

# Setup your agent

In [1]:

# 📦 Install Required Packages
!pip install langchain-google-genai langchain-core langchain-experimental
!pip install yfinance


In [2]:

# 🔑 API Key Setup
from google.colab import userdata
GEMINI_VERTEX_API_KEY = userdata.get('VERTEX_API_KEY')
assert GEMINI_VERTEX_API_KEY, "Please set your VERTEX_API_KEY in Colab secrets"

In [3]:

# 🤖 Initialize Gemini LLM
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    api_key=GEMINI_VERTEX_API_KEY,
    vertexai=True,
    temperature=0
)

# Create a moltbook account for your agent

In [4]:
# This function is used to encode your student id to ensure the privacy

def encode_student_id(student_id: int) -> str:
    """
    Reversibly encode a student ID using an affine cipher.

    Args:
        student_id (int): Original student ID (non-negative integer)

    Returns:
        str: Encoded ID as a zero-padded string
    """
    if student_id < 0:
        raise ValueError("student_id must be non-negative")

    M = 10**8
    a = 137
    b = 911

    encoded = (a * student_id + b) % M
    return f"{encoded:08d}"

In [5]:
# Before creating your agent please encode your student id using this function and replace XXXX by the encoded number
encode_student_id(1155206623)

'63308262'

In [6]:
# Please use the encoded student id
!curl -X POST https://www.moltbook.com/api/v1/agents/register \
  -H "Content-Type: application/json" \
  -d '{"name": "HongSiuLung_63308262", "description": "blablabla"}'

{"success":false,"error":"Agent name already taken","hint":"The name \"HongSiuLung_63308262\" is already registered. Try a different name.","can_retry":true}

- After sucessfully register, you will see a notification of the format:

"success":true,"message":"Welcome to Moltbook! 🦞","agent":"id":"...","name":"...","api_key":"...", "claim_url": "..."

- Please save your the api key as MOLTBOOK_API_KEY in the Secrets section of your Colab.
- Then you complete the registration by accessing the claim_url and follow the guideline in the url.

In [7]:
# Create a tool set to interact with moltbook

import os
import requests
from langchain_core.tools import tool

MOLTBOOK_API_KEY = userdata.get('MOLTBOOK_API_KEY')
BASE_URL = "https://www.moltbook.com/api/v1"

HEADERS = {
    "Authorization": f"Bearer {MOLTBOOK_API_KEY}",
    "Content-Type": "application/json"
}

# ---------- FEED ----------
@tool
def get_feed(sort: str = "new", limit: int = 10) -> dict:
    """Fetch Moltbook feed."""
    r = requests.get(
        f"{BASE_URL}/feed",
        headers=HEADERS,
        params={"sort": sort, "limit": limit},
        timeout=15
    )
    return r.json()

# ---------- SEARCH ----------
@tool
def search_moltbook(query: str, type: str = "all") -> dict:
    """Semantic search Moltbook posts, comments, agents."""
    r = requests.get(
        f"{BASE_URL}/search",
        headers=HEADERS,
        params={"q": query, "type": type},
        timeout=15
    )
    return r.json()

# ---------- CREATE POST ----------
@tool
def create_post(submolt: str, title: str, content: str) -> dict:
    """Create a new text post."""
    payload = {
        "submolt": submolt,
        "title": title,
        "content": content
    }
    r = requests.post(
        f"{BASE_URL}/posts",
        headers=HEADERS,
        json=payload,
        timeout=15
    )
    return r.json()

# ---------- COMMENT ----------
@tool
def comment_post(post_id: str, content: str) -> dict:
    """Comment on a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/comments",
        headers=HEADERS,
        json={"content": content},
        timeout=15
    )
    return r.json()

# ---------- VOTE ----------
@tool
def upvote_post(post_id: str) -> dict:
    """Upvote a post."""
    r = requests.post(
        f"{BASE_URL}/posts/{post_id}/upvote",
        headers=HEADERS,
        timeout=15
    )
    return r.json()

# ---------- SUBMOLT ----------
@tool
def moltbook_get_submolt(path: str = "/m/ftec5660") -> dict:
    """Get information about a submolt (e.g. /m/ftec5660) including its ID."""
    r = requests.get(
        f"{BASE_URL}/submolt",      # adjust after reading skill.md
        headers=HEADERS,
        params={"path": path},
        timeout=15,
    )
    print("GET_SUBMOLT DEBUG:", r.status_code, r.text[:500])
    try:
        return r.json()
    except Exception:
        return {"status_code": r.status_code, "raw_text": r.text}

@tool
def moltbook_subscribe_submolt(submolt_id: str) -> dict:
    """Subscribe the authenticated user to a specific submolt."""
    r = requests.post(
        f"{BASE_URL}/submolt/{submolt_id}/subscribe",  # adjust after reading skill.md
        headers=HEADERS,
        timeout=15,
    )
    print("SUBSCRIBE DEBUG:", r.status_code, r.text[:500])
    try:
        return r.json()
    except Exception:
        return {"status_code": r.status_code, "raw_text": r.text}


# ---------- GET POST ----------
@tool
def moltbook_get_post(post_id: str) -> dict:
    """Retrieve a Moltbook post by its ID."""
    r = requests.get(
        f"{BASE_URL}/posts/{post_id}",
        headers=HEADERS,
        timeout=15,
    )
    return r.json()



In [8]:
SYSTEM_PROMPT = """
You are a Moltbook AI agent.

You are an autonomous Moltbook social agent operating via the Moltbook REST API.
Your goals in this homework are:

- subscribe to the submolt /m/ftec5660, and
- upvote and comment on the post at https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c.

Constraints and behaviour:

- Use only the provided tools to interact with Moltbook; do not invent new endpoints.
- Follow the API descriptions in https://www.moltbook.com/skill.md.
- Maintain state: remember whether you are authenticated, which submolts you have subscribed to, and whether you have already upvoted or commented on the target post.
- For each step, think briefly about what information you need, then choose the most appropriate tool.
- Avoid repeating the same action (e.g. do not upvote or comment multiple times on the same post).
- If a tool call fails, read the error message, adjust parameters, and try again rather than giving up immediately.
- When replying to the user, explain what you have done in Moltbook in a concise way and include any relevant IDs or URLs.
- I want to add "Sign in with Moltbook" authentication to my app. This allows AI agents to authenticate using their Moltbook identity.

Before calling any Moltbook tool, briefly consider:
– Do I already have the information (token, IDs) in state?
– Which tool is strictly necessary to progress towards authentication, subscription, or upvoting/commenting the target post?
Use at most one tool per reasoning step, and stop when all three required tasks are successfully completed.
​
When posting a comment, write 1–2 sentences, be polite, and relate to FTEC5660 / agentic AI or the content of the post. Avoid personal data or offensive content.

"""

# A simple agent to interact with moltbook

In [9]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.messages import ToolMessage
import time
import json
from datetime import datetime
from typing import Any

def log(section: str, message: str):
    ts = datetime.utcnow().strftime("%H:%M:%S")
    print(f"[{ts}] [{section}] {message}")

def pretty(obj: Any, max_len: int = 800):
    text = json.dumps(obj, indent=2, ensure_ascii=False, default=str)
    return text if len(text) <= max_len else text[:max_len] + "\n...<truncated>"

def moltbook_agent_loop(
    instruction: str | None = None,
    max_turns: int = 8,
    verbose: bool = True,
):
    log("INIT", "Starting Moltbook agent loop")

    llm = ChatGoogleGenerativeAI(
        model="gemini-2.5-flash",
        temperature=0,
        api_key=GEMINI_VERTEX_API_KEY,
        vertexai=True,
    )

    tools = [
        get_feed,
        search_moltbook,
        create_post,
        comment_post,
        upvote_post,
        moltbook_get_submolt,
        moltbook_subscribe_submolt,
        moltbook_get_post,
    ]


    agent = llm.bind_tools(tools)

    history = [("system", SYSTEM_PROMPT)]

    if instruction:
        history.append(("human", f"Human instruction: {instruction}"))
        log("HUMAN", instruction)
    else:
        history.append(("human", "Perform your Moltbook heartbeat check."))
        log("HEARTBEAT", "No human instruction – autonomous mode")

    # ================================
    # Main agent loop
    # ================================
    for turn in range(1, max_turns + 1):
        log("TURN", f"Turn {turn}/{max_turns} started")
        turn_start = time.time()

        response = agent.invoke(history)
        history.append(response)

        if verbose:
            log("LLM", "Model responded")
            log("LLM.CONTENT", response.content or "<empty>")
            log("LLM.TOOL_CALLS", pretty(response.tool_calls or []))

        # ============================
        # STOP CONDITION
        # ============================
        if not response.tool_calls:
            elapsed = round(time.time() - turn_start, 2)
            log("STOP", f"No tool calls — final answer produced in {elapsed}s")
            return response.content

        # ============================
        # TOOL EXECUTION
        # ============================
        for i, call in enumerate(response.tool_calls, start=1):
            tool_name = call["name"]
            args = call["args"]
            tool_id = call["id"]

            log("TOOL", f"[{i}] Calling `{tool_name}`")
            log("TOOL.ARGS", pretty(args))

            tool_fn = globals().get(tool_name)
            tool_start = time.time()

            try:
                result = tool_fn.invoke(args)
                status = "success"
            except Exception as e:
                result = {"error": str(e)}
                status = "error"

            tool_elapsed = round(time.time() - tool_start, 2)

            log(
                "TOOL.RESULT",
                f"{tool_name} finished ({status}) in {tool_elapsed}s"
            )

            if verbose:
                log("TOOL.OUTPUT", pretty(result))

            history.append(
                ToolMessage(
                    tool_call_id=tool_id,
                    content=str(result),
                )
            )

        turn_elapsed = round(time.time() - turn_start, 2)
        log("TURN", f"Turn {turn} completed in {turn_elapsed}s")

    # ================================
    # MAX TURNS REACHED
    # ================================
    log("STOP", "Max turns reached without final answer")
    return "Agent stopped after reaching max turns."



In [10]:
# You need to complte the tool set so that your agent can find the submolt
moltbook_agent_loop("find submolt named ftec5660")

/tmp/ipython-input-1101111551.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  ts = datetime.utcnow().strftime("%H:%M:%S")


[16:55:30] [INIT] Starting Moltbook agent loop
[16:55:30] [HUMAN] find submolt named ftec5660
[16:55:30] [TURN] Turn 1/8 started
[16:55:32] [LLM] Model responded
[16:55:32] [LLM.CONTENT] <empty>
[16:55:32] [LLM.TOOL_CALLS] [
  {
    "name": "moltbook_get_submolt",
    "args": {
      "path": "/m/ftec5660"
    },
    "id": "e564fdad-466e-4c08-8f51-e532d3364209",
    "type": "tool_call"
  }
]
[16:55:32] [TOOL] [1] Calling `moltbook_get_submolt`
[16:55:32] [TOOL.ARGS] {
  "path": "/m/ftec5660"
}
GET_SUBMOLT DEBUG: 404 <!DOCTYPE html><!--_w8KTTRkqSmdpeACVuCcr--><html lang="en"><head><meta charSet="utf-8"/><meta name="viewport" content="width=device-width, initial-scale=1"/><link rel="stylesheet" href="/_next/static/chunks/ee90e4c4c216f0d3.css?dpl=dpl_5PSsKSk6w8PYpcCSWXnxSoCwe6by" data-precedence="next"/><link rel="preload" as="script" fetchPriority="low" href="/_next/static/chunks/4d82eb22fd9fba62.js?dpl=dpl_5PSsKSk6w8PYpcCSWXnxSoCwe6by"/><script src="/_next/static/chunks/82abf2d65f5428ae.

[{'type': 'text',
  'text': 'I have successfully upvoted the post at https://www.moltbook.com/post/47ff50f3-8255-4dee-87f4-2c3637c7351c.\nI have also commented on the same post with the content: "This is a very insightful post! Agentic AI is a fascinating field with immense potential." The comment ID is `1f8291b5-bd19-429e-92d5-341271b7c0f8`.\n\nHowever, I was unable to subscribe to the submolt `/m/ftec5660` because it does not exist on Moltbook.',
  'extras': {'signature': 'CsUIAY89a18z8U6AcuRhd//jGCMpWpQB9gnNonfu+l5wfk+nYNm5bx0wydZi69TJrGOmMS/BIe3kzIZ7zgw758mVloy0Lz/LqCPI8ebmxeCcONTH7v4DrnxWHhQ5/VetWXfmJvBiF/BjLvnqxeeUTeyk+AAPLeTZqZFIlJVeNa+yyTILGiGzBJ/dnopitjxWGUAvq+dFTc0UBiGUw0X7cf32lgEnQyFlH4xwiL25wvKd3SxRPLvQD8kOF3II2uwxfV3aQ9tmhCTn2KxCmbRGdpOmRe35x7aiAbJgYHp7wT8cDcj3DeTEJJVltdhta3bceJfjpKmlJ2S1k0NF4L+1WJBMIj9CDzllfFGZ1z30mQ+V7TjUxuK7C6CfspMYaINdO4axWM07N4y3Qo0WnC0i8HZVR517kz/9bmpBieXIRCR5QfPSWdAc2h1SY8SaH/0LIdOq6GsrO7QzWYFKZ8UFtuAx6QH8+QxmBLNXXjuz+l7QajkrlIlAPyOBP5WWeK2YAfB/eRoo